# 7. Managing Growing Projects with Packages, Crates, and Modules

Rust has a number of features that allow you to manage your code’s organization, including which details are exposed, which details are private, and what names are in each scope in your programs. These features, sometimes collectively referred to as the *module system*, include:

- **Packages**: A Cargo feature that lets you build, test, and share crates
- **Crates**: A tree of modules that produces a library or executable
- **Modules** and **use**: Let you control the organization, scope, and privacy of paths
- **Paths**: A way of naming an item, such as a struct, function, or module

## 7.1. Packages and Crates

Crate:

- A `crate` is the smallest amount of code that the Rust compiler considers at a time.
- A `crate` can come in one of two forms:
    - a binary crate: executable to be compiled on, must have the `main` function.
    - a library crate: define functionality intended to be shared with multiple projects.
- The `crate root` is a source file that the Rust compiler starts from and makes up the root module of a crate.

Packages:

- A `package` is a bundle of one or more crates that provides a set of functionality.
- A `package` contains a `Cargo.toml` file that describes how to build those crates.
- A `package` can contain as many binary crates as you like, but **at most only one library crate**.

Walk through when creating a package:
```bash
$ cargo new my-project
     Created binary (application) `my-project` package
$ ls my-project
Cargo.toml
src
$ ls my-project/src
main.rs

```

## 7.2. Defining Modules to Control Scope and Privacy

- module system, `namely paths`` that allow you to name items
- The `use` keyword that brings a path into scope
- The `pub` keyword to make items public
- `as` keyword, external packages, the glob operator.

### Modules Cheat Sheet

- **Start from the crate root** : When compiling a crate, the compiler first looks in the crate root file.
  - src/lib.rs for lib crate
  - src/main.rs for binary crate
- **Declaring modules** : In the crate root file, you can declare new modules. say with `mod garden;`, the compiler will look for:
  - inline, within curly brackets that replace the semicolon following `mod garden`
  - in the file `src/garden.rs`
  - in the file `src/garden/mod.rs`
- **Declaring submodules**: In any file other than the crate root, you can declare submodules. e.x. `mod vegetables;` in `src/garden.rs`:
  - Inline, directly following `mod vegetables`, within curly brackets instead of the semicolon
  - in the file `src/garden/vegetables.rs`
  - `src/garden/vegetables/mod.rs`
- **Paths to code in modules**: Once a module is part of your crate, you can refer to code in that module from anywhere else in that same crate.
  For example, an `Asparagus` type in the garden vegetables module would be found at `crate::garden::vegetables::Asparagus`
- **Private vs public**: Code within a module is private from its parent modules by default.
- **The `use` keyword**: you can create a shortcut with `use crate::garden::vegetables::Asparagus;`, then could write only `Asparagus`.


Here create a sample `backyard` crate:
```

├── Cargo.toml
└── src
    ├── garden
    │   └── vegetables.rs
    ├── garden.rs
    └── main.rs
```

"src/main.rs" : the root file of the binary crate.
```rust
use crate::garden::vegetables::Asparagus; // import Asparagus from -->garden-->vegetables

pub mod garden; // public garden mode --> garden.rs or garden/mod.rs

fn main() {
  let plant = Asparagus {};
  println!("I'm growing {:?}", plant);
}
```

"src/garden.rs": garden module, could also be "src/garden/mod.rs"
```rust
pub mod vegetables; // publib module vegetables, --> vegetables.rs or vegetables/mode.rs
```

"src/vegetables.rs": vegetables module, could also be "src/garden/vegetables/mod.rs"
```rust
#[derive(Debug)]
pub struct Asparagus {}
```

### Grouping Related Code in Modules

`Modules` let us organize code within a crate for readability and easy reuse.

`cargo new restaurant --lib` to create lib crate.

"src/lib.rs":

```rust
mod front_of_house {
  mod hosting {
    fn add_to_waitlist() {}
    fn seat_at_table() {}
  }

  mod serving {
    fn take_order() {}
    fn serve_order() {}
    fn take_payment() {}
  }
}
```

Earlier, we mentioned that `src/main.rs` and `src/lib.rs` are called crate roots. The reason for their name is that the contents of either of these two files form a module named `crate` at the root of the crate’s module structure, known as the `module tree`:

```
crate
 └── front_of_house
     ├── hosting
     │   ├── add_to_waitlist
     │   └── seat_at_table
     └── serving
         ├── take_order
         ├── serve_order
         └── take_payment
```

## 7.3. Paths for Referring to an Item in the Module Tree

We use a path in the same way we use a path when navigating a filesystem. To call a function, we need to know its path.

- An `absolute path` is the full path starting from a crate root.
- A `relative path` starts from the current module and uses `self`, `super`, or an identifier in the current module.

```rust
mod front_of_house {
  pub mod hosting { // pub makes it to be accessable
    pub fn add_to_waitlist() {}
  }
}

pub fn eat_at_restaurant() {
  // absolute path
  crate::front_of_house::hosting::add_to_waitlist();

  // relative path
  front_of_house::hosting::add_to_waitlist();
}
```

### Best Practices for Packages with a Binary and a Library

We mentioned a package can contain both a src/main.rs binary crate root as well as a src/lib.rs library crate root, and both crates will have the package name by default.

The `module tree` should be defined in `src/lib.rs`, then, any public items can be used in the binary crate by
starting paths wit hthe name of the package. The binary crate becomes a user of the library crate just like a completely external crate.

### Starting Relative Paths with super

We can construct relative paths that begin in the parent module, rather than the current module or the crate root, by using `super` at the start of the path. Which works like he `..` syntax in a filesystem.

Using `super` allows us to reference an item that knonw in parent module, which can make rearranging the model tree easier when the moldule is closely related to the parent.

```rust
fn deliver_order() {} // in parent
mod back_of_house {
  fn fix_incorrect_order() {
    cook_order();
    super::deliver_order(); // find in parent
  }
  fn cook_order() {}
}
```

### Making Structs and Enums Public

We can also use `pub`` to designate structs and enums as public, but with a few extra details.

If we use `pub` before a `struct` definition, we make the struct public, but the **struct’s fields will still be private**. We can make each field public or not on a case-by-case basis.


```rust
mod back_of_house {

  pub struct Breakfast {
    pub toast: String,
    seasonal_fruit: String,
  }

  impl Breakfast {
    pub fn summer(toast: &str) -> Breakfast {
      Breakfast {
        toast: String::from(toast),
        seasonal_fruit: String::from("peaches"),
      }
    }
  }
}

pub fn eat_at_restaurant() {
  let mut meal = back_of_house::Breakfast::summer("Rye");
  meal.toast = String::from("Wheat");
  println!("I'd like {} toast please", meal.toast);

  // the below action won't work as the 'seasonal_fruit' was not made into public
  // meal.seasonal_fruit = String::from("blueberries");
}
```

For `enum`, all of its variants are public if we make an enum public.
```rust
mod back_of_house {
    pub enum Appetizer {
        Soup,
        Salad,
    }
}

pub fn eat_at_restaurant() {
    let order1 = back_of_house::Appetizer::Soup;
    let order2 = back_of_house::Appetizer::Salad;
}
```


## 7.4. Bringing Paths into Scope with the use Keyword

we can create a shortcut to a path with the `use` keyword once, and then use the shorter name everywhere else in the scope.

```rust
mod front_of_house {
    pub mod hosting {
        pub fn add_to_waitlist() {}
    }
}

use crate::front_of_house::hosting;

pub fn eat_at_restaurant() {
    hosting::add_to_waitlist();
}
```

Note that `use` only creates the shortcut for the particular scope in which the `use` occurs. When moving `eat_at_restaurant` into a new child module names `customer`, we need to change the `use` location.
```rust
mod front_of_house {
    pub mod hosting {
        pub fn add_to_waitlist() {}
    }
}

mod customer {
    use crate::front_of_house::hosting; // here or
    // use super::hosting // super means from parent root

    pub fn eat_at_restaurant() {
        hosting::add_to_waitlist();
    }
}
```

### Creating Idiomatic use Paths

Specifying the parent module when calling the function makes it clear that the function isn’t locally defined while still minimizing repetition of the full path.

On the other hand, when bringing in structs, enums, and other items with use, it’s idiomatic to specify the full path.
```rust
use std::collections::HashMap;

fn main() {
    let mut map = HashMap::new();
    map.insert(1, 2);
}
```

### Providing New Names with the as Keyword
There’s another solution to the problem of bringing two types of the same name into the same scope with use: after the path, we can specify as and a new local name, or alias, for the type.
```rust
use std::fmt::Result;
use std::io::Result as IoResult;

fn function1() -> Result {
    // --snip--
}

fn function2() -> IoResult<()> {
    // --snip--
}
```

### Re-exporting Names with `pub use`
The imported module will be private by default with `use`. We could use `pub use` to bring the item into scope and also make the item available for others to bring into their scope.

Before this change, external code would have to call the `add_to_waitlist` function by using the path `restaurant::front_of_house::hosting::add_to_waitlist()`. Now that this pub use has re-exported the hosting module from the root module, external code can now use the path `restaurant::hosting::add_to_waitlist()` instead.

```rust
mod front_of_house {
    pub mod hosting {
        pub fn add_to_waitlist() {}
    }
}

pub use crate::front_of_house::hosting;

pub fn eat_at_restaurant() {
    hosting::add_to_waitlist();
}
```

### Using External Packages
As in the 'guessing_game', to use `rand` in our project, we add it into `Cargo.toml`, which tells cargo to download it.
```toml
rand = "0.8.5"
```

To bring `rand` into the scope of our package, we need to `use` it.
```rust
use rand::Rng;

fn main() {
    let secret_number = rand::thread_rng().gen_range(1..=100);
}
```

### Best Practices
```rust
use std::io::{self, Write};
use std::collections::*;
```


## 7.5. Separating Modules into Different Files
```rust
// src/lib.rs
mod front_of_house;

pub use crate::front_of_house::hosting;

pub fn eat_at_restaurant() {
    hosting::add_to_waitlist();
}

// src/front_of_house.rs
pub mod hosting {
    pub fn add_to_waitlist() {}
}
```